In [1]:
print('[Info]: Importing Libraries')

import numpy as np
import pandas as pd
import lightgbm as lgb
from pandas import DataFrame
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

[Info]: Importing Libraries


In [2]:
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
# Helper Functions

def display_missing_data(data_frame):
    return data_frame.isnull().sum()/len(data_frame)*100

def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold:
                colname = corr_matrix.columns[i]
                col_corr.add(colname)
    return col_corr

In [4]:
print('[Info]: Reading the Data\n')

train_data = pd.read_csv('train.csv')
print('Shape of training data :',train_data.shape)

# test_data = pd.read_csv('test.csv.zip')
# print('Shape of testing data :',test_data.shape)

[Info]: Reading the Data

Shape of training data : (85741, 133)


In [5]:
train_data.head(5)

,IDs,target_label,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100,x101,x102,x103,x104,x105,x106,x107,x108,x109,x110,x111,x112,x113,x114,x115,x116,x117,x118,x119,x120,x121,x122,x123,x124,x125,x126,x127,x128,x129,x130,x131
0,3,1,3.321300,8.880867,1.614988,18.869283,1.588448,1.355013,11.636387,2.857144,17.142857,0.095678,3.670350,B,11.784549,5.909091,1.989780,1.083033,8.000000,1.335739,18.303925,0.803572,15.231789,8.571429,0.636365,8.861647,0,10.890984,AU,0.649820,0.905342,0.037096,3.517720,C,0.454546,0.139412,9.314079,2.015505,G,7.277792,3.000000,1,C,2.230940,8.877414,15.858152,5.814018,3.971118,1.707317,8.086643,3.176895,12.707581,7.711453,NaN,16.434108,XDX,12.579184,C,B,1.059603,4.374296,16.747968,11.326592,0.035754,0,5.454545,0.007294,9.848672,0.113561,2.857144,2.382692,-6.297423e-07,8.375452,9.999999,O,2.599278,7.915266,E,1.574039,E,C,0.529802,1.299638,12.171733,2.866830,A,7.436824,7.292418,1.720818,19.470199,1.951220,0.503281,3.930922,7.653429,9.551836,0.442252,1.804126,A,2.024285,1,0.866426,15.634907,D,4.012088,0.106720,F,0.433213,7.730923,10.498338,0.153461,0.462019,3.921026,6.085711,0.148883,6.363189,6.592012,0.012941,-1.716131e-08,1.010829,7.528326,7.270147,DI,0.590122,1.191337,C,8.571429,3.113719,3.393503,0.899420,2.757375,8.727474,8.389237,1.299638
1,4,1,NaN,NaN,NaN,NaN,NaN,NaN,11.636386,NaN,NaN,2.678584,NaN,A,12.053353,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.430691,0,NaN,AF,NaN,NaN,1.129469,NaN,A,NaN,3.056144,NaN,NaN,G,NaN,NaN,2,C,NaN,8.303967,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.305766,G,NaN,GUV,NaN,C,B,NaN,NaN,NaN,14.579479,0.598896,0,NaN,1.505335,NaN,2.449959,NaN,1.825361,NaN,NaN,NaN,U,NaN,9.191265,B,NaN,D,E,NaN,NaN,NaN,NaN,B,NaN,NaN,NaN,NaN,NaN,1.312910,4.247858,NaN,9.848004,NaN,NaN,A,1.957825,2,NaN,10.308044,D,NaN,NaN,F,NaN,6.763110,NaN,2.544736,NaN,NaN,6.507647,NaN,NaN,10.595357,2.301630,NaN,NaN,7.277655,3.615077,DY,NaN,NaN,C,NaN,NaN,NaN,1.379210,NaN,NaN,NaN,NaN
2,6,1,1.408046,8.874521,2.272541,18.376407,1.229246,1.945044,14.094723,10.574713,9.425288,0.039051,3.610789,B,13.594728,7.180722,1.812795,1.587644,7.018256,0.797415,17.011645,1.956521,18.568129,5.517242,1.677108,13.315819,0,11.777912,CJ,1.681034,1.042425,0.139721,3.423944,A,0.433735,0.148843,9.703065,1.470878,H,4.780357,1.947261,1,C,2.188198,6.942002,15.927390,5.498902,3.965517,1.587045,8.591954,1.987549,12.935823,11.523045,NaN,16.347483,ACUE,12.554274,C,B,1.166281,4.872157,16.621695,11.302795,0.002267,1,6.746988,0.113967,9.670823,0.108387,1.034483,2.230754,5.655086e-01,8.898468,8.965516,J,2.097700,11.627438,B,2.573664,B,C,1.732102,1.178161,12.194855,3.903302,B,7.375480,8.213602,1.308269,18.256352,1.511063,6.542669,1.990131,7.931035,10.747144,0.763925,1.415230,B,1.990847,2,1.242817,13.777666,D,0.287322,1.224114,F,2.643678,7.517125,12.708574,0.140260,0.832518,4.225930,9.646653,0.231630,6.292979,4.949609,0.171947,-4.805344e-07,1.666667,4.797873,8.703550,BW,8.926662,1.334611,D,13.448277,2.954381,2.303640,3.329176,2.503055,8.304757,8.507281,1.379310
3,8,1,NaN,NaN,NaN,NaN,NaN,NaN,10.991098,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,Z,NaN,NaN,NaN,NaN,C,NaN,NaN,NaN,NaN,H,NaN,NaN,1,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.138920,G,NaN,HIT,NaN,NaN,B,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T,NaN,NaN,C,NaN,C,I,NaN,NaN,NaN,NaN,G,NaN,NaN,NaN,NaN,NaN,1.050328,NaN,NaN,NaN,NaN,NaN,A,NaN,1,NaN,14.097099,D,NaN,NaN,F,NaN,6.414567,NaN,NaN,NaN,NaN,6.320087,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.083151,NaN,NaN,NaN,E,NaN,NaN,NaN,1.364536,NaN,NaN,NaN,NaN
4,12,0,1.147860,7.928015,2.519063,17.226675,0.980870,1.968303,12.148604,8.543690,11.456310,0.075958,2.873974,B,14.152569,6.593406,1.992594,1.619651,3.4762

In [6]:
train_data.target_label.value_counts()

1    65266
0    20475
Name: target_label, dtype: int64

In [7]:
print('[Info]: Separating Depending and Independent Varibales')

X = train_data.drop(columns=['target_label', 'IDs'],axis=1)
y = train_data['target_label']

[Info]: Separating Depending and Independent Varibales


In [8]:
print('[Info]: Separating Numerical Data \n')

numerical_data = X.select_dtypes(include=[np.number])
print('Shape of numerical data :', numerical_data.shape)

[Info]: Separating Numerical Data 

Shape of numerical data : (85741, 112)


In [9]:
display_missing_data(numerical_data)

x1      43.599911
x2      43.599911
x3      43.603410
x4      43.608075
x5      43.599911
          ...    
x127     0.075810
x128    44.867683
x129    43.561423
x130    43.561423
x131    43.599911
Length: 112, dtype: float64

### Modelling using Numerical Data only with Imputing Value -999  

In [10]:
df_const_imputed = numerical_data.fillna(-999)

In [11]:
df_const_imputed.head(5)

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x28,x29,x30,x31,x33,x34,x35,x36,x38,x39,x40,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x53,x55,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x74,x75,x77,x80,x81,x82,x83,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x97,x98,x99,x100,x102,x103,x105,x106,x107,x108,x109,x110,x111,x112,x113,x114,x115,x116,x117,x118,x119,x121,x122,x124,x125,x126,x127,x128,x129,x130,x131
0,3.321300,8.880867,1.614988,18.869283,1.588448,1.355013,11.636387,2.857144,17.142857,0.095678,3.670350,11.784549,5.909091,1.989780,1.083033,8.000000,1.335739,18.303925,0.803572,15.231789,8.571429,0.636365,8.861647,0,10.890984,0.649820,0.905342,0.037096,3.517720,0.454546,0.139412,9.314079,2.015505,7.277792,3.000000,1,2.230940,8.877414,15.858152,5.814018,3.971118,1.707317,8.086643,3.176895,12.707581,7.711453,16.434108,12.579184,1.059603,4.374296,16.747968,11.326592,0.035754,0,5.454545,0.007294,9.848672,0.113561,2.857144,2.382692,-6.297423e-07,8.375452,9.999999,2.599278,7.915266,1.574039,0.529802,1.299638,12.171733,2.866830,7.436824,7.292418,1.720818,19.470199,1.951220,0.503281,3.930922,7.653429,9.551836,0.442252,1.804126,2.024285,1,0.866426,15.634907,4.012088,0.106720,0.433213,7.730923,10.498338,0.153461,0.462019,3.921026,6.085711,0.148883,6.363189,6.592012,0.012941,-1.716131e-08,1.010829,7.528326,7.270147,0.590122,1.191337,8.571429,3.113719,3.393503,0.899420,2.757375,8.727474,8.389237,1.299638
1,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,11.636386,-999.000000,-999.000000,2.678584,-999.000000,12.053353,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,3.430691,0,-999.000000,-999.000000,-999.000000,1.129469,-999.000000,-999.000000,3.056144,-999.000000,-999.000000,-999.000000,-999.000000,2,-999.000000,8.303967,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,14.305766,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,14.579479,0.598896,0,-999.000000,1.505335,-999.000000,2.449959,-999.000000,1.825361,-9.990000e+02,-999.000000,-999.000000,-999.000000,9.191265,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,1.312910,4.247858,-999.000000,9.848004,-999.000000,-999.000000,1.957825,2,-999.000000,10.308044,-999.000000,-999.000000,-999.000000,6.763110,-999.000000,2.544736,-999.000000,-999.000000,6.507647,-999.000000,-999.000000,10.595357,2.301630,-9.990000e+02,-999.000000,7.277655,3.615077,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,1.379210,-999.000000,-999.000000,-999.000000,-999.000000
2,1.408046,8.874521,2.272541,18.376407,1.229246,1.945044,14.094723,10.574713,9.425288,0.039051,3.610789,13.594728,7.180722,1.812795,1.587644,7.018256,0.797415,17.011645,1.956521,18.568129,5.517242,1.677108,13.315819,0,11.777912,1.681034,1.042425,0.139721,3.423944,0.433735,0.148843,9.703065,1.470878,4.780357,1.947261,1,2.188198,6.942002,15.927390,5.498902,3.965517,1.587045,8.591954,1.987549,12.935823,11.523045,16.347483,12.554274,1.166281,4.872157,16.621695,11.302795,0.002267,1,6.746988,0.113967,9.670823,0.108387,1.034483,2.230754,5.655086e-01,8.898468,8.965516,2.097700,11.627438,2.573664,1.732102,1.178161,12.194855,3.903302,7.375480,8.213602,1.308269,18.256352,1.511063,6.542669,1.990131,7.931035,10.747144,0.763925,1.415230,1.990847,2,1.242817,13.777666,0.287322,1.224114,2.643678,7.517125,12.708574,0.140260,0.832518,4.225930,9.646653,0.231630,6.292979,4.949609,0.171947,-4.805344e-07,1.666667,4.797873,8.703550,8.926662,1.334611,13.448277,2.954381,2.303640,3.329176,2.503055,8.304757,8.507281,1.379310
3,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,10.991098,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000

In [12]:
#Now splitting our dataset into test and train 

X_train, X_test, y_train, y_test = train_test_split(df_const_imputed, y,
                                                 test_size=.3, random_state=0)
X_train.shape, X_test.shape

((60018, 112), (25723, 112))

In [13]:
trn_data = lgb.Dataset(X_train, label = y_train)
val_data = lgb.Dataset(X_test, label = y_test)

In [14]:
#setting parameters for lightgbm
param = {'num_leaves':30, 'objective':'binary','max_depth':7,
                 'learning_rate':.005,'max_bin':50, 'metric':'binary_logloss',
                 'verbose': -1, 'lambda_l1': 0.1}

In [15]:
#training our model using light gbm
num_round = 500
start=datetime.now()
lgbm=lgb.train(param, train_set = trn_data, num_boost_round = num_round,
               valid_sets=val_data, early_stopping_rounds = 50, verbose_eval = True)
stop=datetime.now()
execution_time_lgbm = stop-start
execution_time_lgbm

[1]	valid_0's binary_logloss: 0.547171
Training until validation scores don't improve for 50 rounds
[2]	valid_0's binary_logloss: 0.54666
[3]	valid_0's binary_logloss: 0.546155
[4]	valid_0's binary_logloss: 0.545653
[5]	valid_0's binary_logloss: 0.545159
[6]	valid_0's binary_logloss: 0.544668
[7]	valid_0's binary_logloss: 0.544185
[8]	valid_0's binary_logloss: 0.543707
[9]	valid_0's binary_logloss: 0.543233
[10]	valid_0's binary_logloss: 0.542765
[11]	valid_0's binary_logloss: 0.542305
[12]	valid_0's binary_logloss: 0.54185
[13]	valid_0's binary_logloss: 0.541396
[14]	valid_0's binary_logloss: 0.540952
[15]	valid_0's binary_logloss: 0.540512
[16]	valid_0's binary_logloss: 0.540075
[17]	valid_0's binary_logloss: 0.539644
[18]	valid_0's binary_logloss: 0.539215
[19]	valid_0's binary_logloss: 0.538793
[20]	valid_0's binary_logloss: 0.538373
[21]	valid_0's binary_logloss: 0.537961
[22]	valid_0's binary_logloss: 0.537547
[23]	valid_0's binary_logloss: 0.537145
[24]	valid_0's binary_logloss:

[211]	valid_0's binary_logloss: 0.501517
[212]	valid_0's binary_logloss: 0.501433
[213]	valid_0's binary_logloss: 0.50135
[214]	valid_0's binary_logloss: 0.501273
[215]	valid_0's binary_logloss: 0.501199
[216]	valid_0's binary_logloss: 0.501118
[217]	valid_0's binary_logloss: 0.501041
[218]	valid_0's binary_logloss: 0.500974
[219]	valid_0's binary_logloss: 0.500901
[220]	valid_0's binary_logloss: 0.50083
[221]	valid_0's binary_logloss: 0.500761
[222]	valid_0's binary_logloss: 0.500683
[223]	valid_0's binary_logloss: 0.500607
[224]	valid_0's binary_logloss: 0.500536
[225]	valid_0's binary_logloss: 0.500474
[226]	valid_0's binary_logloss: 0.500396
[227]	valid_0's binary_logloss: 0.500321
[228]	valid_0's binary_logloss: 0.500253
[229]	valid_0's binary_logloss: 0.500188
[230]	valid_0's binary_logloss: 0.500115
[231]	valid_0's binary_logloss: 0.500039
[232]	valid_0's binary_logloss: 0.499966
[233]	valid_0's binary_logloss: 0.499897
[234]	valid_0's binary_logloss: 0.499831
[235]	valid_0's bi

[418]	valid_0's binary_logloss: 0.493293
[419]	valid_0's binary_logloss: 0.493274
[420]	valid_0's binary_logloss: 0.493259
[421]	valid_0's binary_logloss: 0.493243
[422]	valid_0's binary_logloss: 0.493222
[423]	valid_0's binary_logloss: 0.493207
[424]	valid_0's binary_logloss: 0.493191
[425]	valid_0's binary_logloss: 0.493176
[426]	valid_0's binary_logloss: 0.493161
[427]	valid_0's binary_logloss: 0.49314
[428]	valid_0's binary_logloss: 0.493126
[429]	valid_0's binary_logloss: 0.493106
[430]	valid_0's binary_logloss: 0.493089
[431]	valid_0's binary_logloss: 0.493076
[432]	valid_0's binary_logloss: 0.493061
[433]	valid_0's binary_logloss: 0.49304
[434]	valid_0's binary_logloss: 0.493018
[435]	valid_0's binary_logloss: 0.493004
[436]	valid_0's binary_logloss: 0.492988
[437]	valid_0's binary_logloss: 0.492971
[438]	valid_0's binary_logloss: 0.492956
[439]	valid_0's binary_logloss: 0.492939
[440]	valid_0's binary_logloss: 0.492921
[441]	valid_0's binary_logloss: 0.492908
[442]	valid_0's bi

datetime.timedelta(seconds=5, microseconds=722502)

In [16]:
#predicting on train set
y_pred_lbg_train = lgbm.predict(X_train)
y_pred_lbg_train = np.where(y_pred_lbg_train > .5, 1 , 0)

#predicting on test set
y_pred_lbg = lgbm.predict(X_test)
y_pred_lbg = np.where(y_pred_lbg > .5, 1 , 0)

In [17]:
#calculating accuracy
print('Train accuracy is {0:0.4} %' .format(accuracy_score(y_train,y_pred_lbg_train)))
print('Test accuracy is {0:0.4} %' .format(accuracy_score(y_test,y_pred_lbg)))

Train accuracy is 0.7749 %
Test accuracy is 0.7712 %


In [18]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred_lbg)
print('Confusion matrix Validation Data\n\n', cm)

Confusion matrix Validation Data

 [[  632  5466]
 [  420 19205]]


In [19]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_train, y_pred_lbg_train)
print('Confusion matrix Train Data\n\n', cm)

Confusion matrix Train Data

 [[ 1721 12656]
 [  853 44788]]


### Imputing Mean

In [20]:
#Now splitting our dataset into test and train 

X_train, X_test, y_train, y_test = train_test_split(numerical_data, y,
                                                 test_size=.3, random_state=0)
X_train.shape, X_test.shape

((60018, 112), (25723, 112))

In [21]:
# Imputing Mean from Train data to Train and Test Data
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_train.mean())

In [22]:
train_data = lgb.Dataset(X_train, label = y_train)
val_data = lgb.Dataset(X_test, label = y_test)

In [23]:
#setting parameters for lightgbm
param = {'num_leaves':30, 'objective':'binary','max_depth':7,
                 'learning_rate':.005,'max_bin':50, 'metric':'binary_logloss',
                 'verbose': -1, 'lambda_l1': 0.1}

In [24]:
#training our model using light gbm
num_round = 1500
start = datetime.now()
lgbm = lgb.train(param, train_set=train_data, num_boost_round= num_round,
                 valid_sets= val_data, early_stopping_rounds = 10, verbose_eval = True)
stop = datetime.now()
execution_time_lgbm = stop-start
execution_time_lgbm

[1]	valid_0's binary_logloss: 0.547166
Training until validation scores don't improve for 10 rounds
[2]	valid_0's binary_logloss: 0.546649
[3]	valid_0's binary_logloss: 0.546139
[4]	valid_0's binary_logloss: 0.545637
[5]	valid_0's binary_logloss: 0.545143
[6]	valid_0's binary_logloss: 0.544655
[7]	valid_0's binary_logloss: 0.544169
[8]	valid_0's binary_logloss: 0.543692
[9]	valid_0's binary_logloss: 0.54322
[10]	valid_0's binary_logloss: 0.542748
[11]	valid_0's binary_logloss: 0.542287
[12]	valid_0's binary_logloss: 0.541827
[13]	valid_0's binary_logloss: 0.541375
[14]	valid_0's binary_logloss: 0.540928
[15]	valid_0's binary_logloss: 0.540482
[16]	valid_0's binary_logloss: 0.540044
[17]	valid_0's binary_logloss: 0.539611
[18]	valid_0's binary_logloss: 0.539179
[19]	valid_0's binary_logloss: 0.538756
[20]	valid_0's binary_logloss: 0.538336
[21]	valid_0's binary_logloss: 0.537924
[22]	valid_0's binary_logloss: 0.537517
[23]	valid_0's binary_logloss: 0.537109
[24]	valid_0's binary_logloss

[213]	valid_0's binary_logloss: 0.501317
[214]	valid_0's binary_logloss: 0.501242
[215]	valid_0's binary_logloss: 0.501166
[216]	valid_0's binary_logloss: 0.501093
[217]	valid_0's binary_logloss: 0.501011
[218]	valid_0's binary_logloss: 0.500937
[219]	valid_0's binary_logloss: 0.500863
[220]	valid_0's binary_logloss: 0.500782
[221]	valid_0's binary_logloss: 0.500709
[222]	valid_0's binary_logloss: 0.500638
[223]	valid_0's binary_logloss: 0.500565
[224]	valid_0's binary_logloss: 0.500496
[225]	valid_0's binary_logloss: 0.500427
[226]	valid_0's binary_logloss: 0.500358
[227]	valid_0's binary_logloss: 0.50029
[228]	valid_0's binary_logloss: 0.500222
[229]	valid_0's binary_logloss: 0.500154
[230]	valid_0's binary_logloss: 0.500074
[231]	valid_0's binary_logloss: 0.500007
[232]	valid_0's binary_logloss: 0.499949
[233]	valid_0's binary_logloss: 0.499883
[234]	valid_0's binary_logloss: 0.499815
[235]	valid_0's binary_logloss: 0.499746
[236]	valid_0's binary_logloss: 0.499686
[237]	valid_0's b

[428]	valid_0's binary_logloss: 0.493056
[429]	valid_0's binary_logloss: 0.49304
[430]	valid_0's binary_logloss: 0.493025
[431]	valid_0's binary_logloss: 0.493007
[432]	valid_0's binary_logloss: 0.492995
[433]	valid_0's binary_logloss: 0.492982
[434]	valid_0's binary_logloss: 0.492964
[435]	valid_0's binary_logloss: 0.49295
[436]	valid_0's binary_logloss: 0.492938
[437]	valid_0's binary_logloss: 0.492922
[438]	valid_0's binary_logloss: 0.49291
[439]	valid_0's binary_logloss: 0.492887
[440]	valid_0's binary_logloss: 0.492875
[441]	valid_0's binary_logloss: 0.49286
[442]	valid_0's binary_logloss: 0.492846
[443]	valid_0's binary_logloss: 0.492823
[444]	valid_0's binary_logloss: 0.492812
[445]	valid_0's binary_logloss: 0.492792
[446]	valid_0's binary_logloss: 0.492771
[447]	valid_0's binary_logloss: 0.492763
[448]	valid_0's binary_logloss: 0.492747
[449]	valid_0's binary_logloss: 0.492734
[450]	valid_0's binary_logloss: 0.492714
[451]	valid_0's binary_logloss: 0.492702
[452]	valid_0's bina

[645]	valid_0's binary_logloss: 0.490551
[646]	valid_0's binary_logloss: 0.490549
[647]	valid_0's binary_logloss: 0.490531
[648]	valid_0's binary_logloss: 0.490522
[649]	valid_0's binary_logloss: 0.490515
[650]	valid_0's binary_logloss: 0.490512
[651]	valid_0's binary_logloss: 0.490512
[652]	valid_0's binary_logloss: 0.490504
[653]	valid_0's binary_logloss: 0.4905
[654]	valid_0's binary_logloss: 0.490484
[655]	valid_0's binary_logloss: 0.490476
[656]	valid_0's binary_logloss: 0.490461
[657]	valid_0's binary_logloss: 0.49046
[658]	valid_0's binary_logloss: 0.490454
[659]	valid_0's binary_logloss: 0.490445
[660]	valid_0's binary_logloss: 0.490428
[661]	valid_0's binary_logloss: 0.490425
[662]	valid_0's binary_logloss: 0.490417
[663]	valid_0's binary_logloss: 0.490401
[664]	valid_0's binary_logloss: 0.490397
[665]	valid_0's binary_logloss: 0.49039
[666]	valid_0's binary_logloss: 0.490384
[667]	valid_0's binary_logloss: 0.490366
[668]	valid_0's binary_logloss: 0.490362
[669]	valid_0's bina

[857]	valid_0's binary_logloss: 0.489546
[858]	valid_0's binary_logloss: 0.489543
[859]	valid_0's binary_logloss: 0.489541
[860]	valid_0's binary_logloss: 0.489539
[861]	valid_0's binary_logloss: 0.489537
[862]	valid_0's binary_logloss: 0.489536
[863]	valid_0's binary_logloss: 0.489528
[864]	valid_0's binary_logloss: 0.489525
[865]	valid_0's binary_logloss: 0.489523
[866]	valid_0's binary_logloss: 0.489519
[867]	valid_0's binary_logloss: 0.489516
[868]	valid_0's binary_logloss: 0.489514
[869]	valid_0's binary_logloss: 0.489516
[870]	valid_0's binary_logloss: 0.489512
[871]	valid_0's binary_logloss: 0.489504
[872]	valid_0's binary_logloss: 0.489503
[873]	valid_0's binary_logloss: 0.4895
[874]	valid_0's binary_logloss: 0.489489
[875]	valid_0's binary_logloss: 0.489487
[876]	valid_0's binary_logloss: 0.489483
[877]	valid_0's binary_logloss: 0.489482
[878]	valid_0's binary_logloss: 0.48948
[879]	valid_0's binary_logloss: 0.489473
[880]	valid_0's binary_logloss: 0.489465
[881]	valid_0's bin

[1075]	valid_0's binary_logloss: 0.48897
[1076]	valid_0's binary_logloss: 0.488967
[1077]	valid_0's binary_logloss: 0.488964
[1078]	valid_0's binary_logloss: 0.488962
[1079]	valid_0's binary_logloss: 0.48896
[1080]	valid_0's binary_logloss: 0.488955
[1081]	valid_0's binary_logloss: 0.488955
[1082]	valid_0's binary_logloss: 0.488954
[1083]	valid_0's binary_logloss: 0.488951
[1084]	valid_0's binary_logloss: 0.488949
[1085]	valid_0's binary_logloss: 0.488949
[1086]	valid_0's binary_logloss: 0.488948
[1087]	valid_0's binary_logloss: 0.488948
[1088]	valid_0's binary_logloss: 0.488946
[1089]	valid_0's binary_logloss: 0.488945
[1090]	valid_0's binary_logloss: 0.488942
[1091]	valid_0's binary_logloss: 0.488941
[1092]	valid_0's binary_logloss: 0.488937
[1093]	valid_0's binary_logloss: 0.488934
[1094]	valid_0's binary_logloss: 0.488931
[1095]	valid_0's binary_logloss: 0.488932
[1096]	valid_0's binary_logloss: 0.488927
[1097]	valid_0's binary_logloss: 0.488919
[1098]	valid_0's binary_logloss: 0.4

[1271]	valid_0's binary_logloss: 0.48868
[1272]	valid_0's binary_logloss: 0.488675
[1273]	valid_0's binary_logloss: 0.488676
[1274]	valid_0's binary_logloss: 0.488675
[1275]	valid_0's binary_logloss: 0.488674
[1276]	valid_0's binary_logloss: 0.488674
[1277]	valid_0's binary_logloss: 0.488672
[1278]	valid_0's binary_logloss: 0.488673
[1279]	valid_0's binary_logloss: 0.488674
[1280]	valid_0's binary_logloss: 0.488671
[1281]	valid_0's binary_logloss: 0.488671
[1282]	valid_0's binary_logloss: 0.488666
[1283]	valid_0's binary_logloss: 0.488666
[1284]	valid_0's binary_logloss: 0.488671
[1285]	valid_0's binary_logloss: 0.488671
[1286]	valid_0's binary_logloss: 0.48867
[1287]	valid_0's binary_logloss: 0.488668
[1288]	valid_0's binary_logloss: 0.488666
[1289]	valid_0's binary_logloss: 0.488664
[1290]	valid_0's binary_logloss: 0.488665
[1291]	valid_0's binary_logloss: 0.488663
[1292]	valid_0's binary_logloss: 0.488663
[1293]	valid_0's binary_logloss: 0.488663
[1294]	valid_0's binary_logloss: 0.4

datetime.timedelta(seconds=13, microseconds=896928)

In [25]:
#predicting on train set
y_pred_lbg_train = lgbm.predict(X_train)

#predicting on test set
y_pred_lbg = lgbm.predict(X_test)


In [26]:
y_pred_lbg_train = np.where(y_pred_lbg_train > .5, 1 , 0)
y_pred_lbg = np.where(y_pred_lbg > .5, 1 , 0)

In [27]:
#calculating accuracy
print('Train accuracy is {0:0.4} ' .format(accuracy_score(y_train, y_pred_lbg_train)))
print('Test accuracy is {0:0.4} ' .format(accuracy_score(y_test, y_pred_lbg)))

Train accuracy is 0.7824 
Test accuracy is 0.7732 


In [28]:
cm = confusion_matrix(y_train, y_pred_lbg_train)
print('Confusion matrix Train Data\n', cm, '\n')

cm = confusion_matrix(y_test, y_pred_lbg)
print('Confusion matrix Validation Data\n', cm)

Confusion matrix Train Data
 [[ 2476 11901]
 [ 1158 44483]] 

Confusion matrix Validation Data
 [[  870  5228]
 [  606 19019]]


### Removing Similar features based on correlaltion

In [29]:
# separate train and test sets

X_train_corr, X_test_corr, y_train, y_test = train_test_split(
    numerical_data, y, test_size = 0.3, random_state = 8)

X_train_corr.shape, X_test_corr.shape

((60018, 112), (25723, 112))

In [30]:
corr_features = correlation(X_train_corr, 0.8)
len(set(corr_features))

58

In [31]:
X_train_corr = X_train_corr.drop(labels = corr_features, axis=1)
X_test_corr = X_test_corr.drop(labels = corr_features, axis=1)
X_train_corr.shape, X_test_corr.shape

((60018, 54), (25723, 54))

In [32]:
train_data = lgb.Dataset(X_train_corr, label = y_train)
val_data = lgb.Dataset(X_test_corr, label = y_test)

In [33]:
#setting parameters for lightgbm
param = {'num_leaves':30, 'objective':'binary','max_depth':7,
                 'learning_rate':.005,'max_bin':50, 'metric':'binary_logloss',
                 'verbose': -1, 'lambda_l1': 0.1}

#training our model using light gbm
num_round = 1500
start = datetime.now()
lgbm = lgb.train(param, train_set = train_data, num_boost_round = num_round,
                 valid_sets = val_data, early_stopping_rounds = 50, verbose_eval = True)
stop = datetime.now()
execution_time_lgbm = stop-start
execution_time_lgbm

[1]	valid_0's binary_logloss: 0.548737
Training until validation scores don't improve for 50 rounds
[2]	valid_0's binary_logloss: 0.548221
[3]	valid_0's binary_logloss: 0.54771
[4]	valid_0's binary_logloss: 0.547207
[5]	valid_0's binary_logloss: 0.546707
[6]	valid_0's binary_logloss: 0.546214
[7]	valid_0's binary_logloss: 0.545724
[8]	valid_0's binary_logloss: 0.545238
[9]	valid_0's binary_logloss: 0.544759
[10]	valid_0's binary_logloss: 0.544283
[11]	valid_0's binary_logloss: 0.543818
[12]	valid_0's binary_logloss: 0.543357
[13]	valid_0's binary_logloss: 0.542899
[14]	valid_0's binary_logloss: 0.542448
[15]	valid_0's binary_logloss: 0.541999
[16]	valid_0's binary_logloss: 0.541558
[17]	valid_0's binary_logloss: 0.541119
[18]	valid_0's binary_logloss: 0.540687
[19]	valid_0's binary_logloss: 0.54026
[20]	valid_0's binary_logloss: 0.539835
[21]	valid_0's binary_logloss: 0.539417
[22]	valid_0's binary_logloss: 0.539002
[23]	valid_0's binary_logloss: 0.538592
[24]	valid_0's binary_logloss:

[220]	valid_0's binary_logloss: 0.501768
[221]	valid_0's binary_logloss: 0.501694
[222]	valid_0's binary_logloss: 0.501621
[223]	valid_0's binary_logloss: 0.501552
[224]	valid_0's binary_logloss: 0.501481
[225]	valid_0's binary_logloss: 0.501409
[226]	valid_0's binary_logloss: 0.501342
[227]	valid_0's binary_logloss: 0.50127
[228]	valid_0's binary_logloss: 0.501205
[229]	valid_0's binary_logloss: 0.501135
[230]	valid_0's binary_logloss: 0.501072
[231]	valid_0's binary_logloss: 0.501005
[232]	valid_0's binary_logloss: 0.500928
[233]	valid_0's binary_logloss: 0.500866
[234]	valid_0's binary_logloss: 0.5008
[235]	valid_0's binary_logloss: 0.50074
[236]	valid_0's binary_logloss: 0.500675
[237]	valid_0's binary_logloss: 0.500615
[238]	valid_0's binary_logloss: 0.500551
[239]	valid_0's binary_logloss: 0.50049
[240]	valid_0's binary_logloss: 0.50043
[241]	valid_0's binary_logloss: 0.500357
[242]	valid_0's binary_logloss: 0.500302
[243]	valid_0's binary_logloss: 0.500243
[244]	valid_0's binary

[431]	valid_0's binary_logloss: 0.493887
[432]	valid_0's binary_logloss: 0.493868
[433]	valid_0's binary_logloss: 0.493856
[434]	valid_0's binary_logloss: 0.493837
[435]	valid_0's binary_logloss: 0.493821
[436]	valid_0's binary_logloss: 0.493808
[437]	valid_0's binary_logloss: 0.493793
[438]	valid_0's binary_logloss: 0.493782
[439]	valid_0's binary_logloss: 0.493758
[440]	valid_0's binary_logloss: 0.493748
[441]	valid_0's binary_logloss: 0.493733
[442]	valid_0's binary_logloss: 0.493716
[443]	valid_0's binary_logloss: 0.493703
[444]	valid_0's binary_logloss: 0.493693
[445]	valid_0's binary_logloss: 0.493685
[446]	valid_0's binary_logloss: 0.493665
[447]	valid_0's binary_logloss: 0.493653
[448]	valid_0's binary_logloss: 0.493628
[449]	valid_0's binary_logloss: 0.493613
[450]	valid_0's binary_logloss: 0.493603
[451]	valid_0's binary_logloss: 0.493588
[452]	valid_0's binary_logloss: 0.493562
[453]	valid_0's binary_logloss: 0.493546
[454]	valid_0's binary_logloss: 0.49353
[455]	valid_0's b

[664]	valid_0's binary_logloss: 0.491499
[665]	valid_0's binary_logloss: 0.491483
[666]	valid_0's binary_logloss: 0.491482
[667]	valid_0's binary_logloss: 0.491475
[668]	valid_0's binary_logloss: 0.49146
[669]	valid_0's binary_logloss: 0.491453
[670]	valid_0's binary_logloss: 0.491453
[671]	valid_0's binary_logloss: 0.491436
[672]	valid_0's binary_logloss: 0.49143
[673]	valid_0's binary_logloss: 0.491425
[674]	valid_0's binary_logloss: 0.49141
[675]	valid_0's binary_logloss: 0.491408
[676]	valid_0's binary_logloss: 0.491401
[677]	valid_0's binary_logloss: 0.491384
[678]	valid_0's binary_logloss: 0.49138
[679]	valid_0's binary_logloss: 0.491367
[680]	valid_0's binary_logloss: 0.491361
[681]	valid_0's binary_logloss: 0.491357
[682]	valid_0's binary_logloss: 0.491341
[683]	valid_0's binary_logloss: 0.491341
[684]	valid_0's binary_logloss: 0.491335
[685]	valid_0's binary_logloss: 0.491319
[686]	valid_0's binary_logloss: 0.491316
[687]	valid_0's binary_logloss: 0.491302
[688]	valid_0's bina

[896]	valid_0's binary_logloss: 0.490424
[897]	valid_0's binary_logloss: 0.490419
[898]	valid_0's binary_logloss: 0.490416
[899]	valid_0's binary_logloss: 0.490415
[900]	valid_0's binary_logloss: 0.490412
[901]	valid_0's binary_logloss: 0.49041
[902]	valid_0's binary_logloss: 0.490406
[903]	valid_0's binary_logloss: 0.490402
[904]	valid_0's binary_logloss: 0.490401
[905]	valid_0's binary_logloss: 0.490395
[906]	valid_0's binary_logloss: 0.490393
[907]	valid_0's binary_logloss: 0.490393
[908]	valid_0's binary_logloss: 0.490393
[909]	valid_0's binary_logloss: 0.490388
[910]	valid_0's binary_logloss: 0.490378
[911]	valid_0's binary_logloss: 0.490378
[912]	valid_0's binary_logloss: 0.490375
[913]	valid_0's binary_logloss: 0.490366
[914]	valid_0's binary_logloss: 0.490364
[915]	valid_0's binary_logloss: 0.490361
[916]	valid_0's binary_logloss: 0.490362
[917]	valid_0's binary_logloss: 0.490359
[918]	valid_0's binary_logloss: 0.490358
[919]	valid_0's binary_logloss: 0.490354
[920]	valid_0's b

[1093]	valid_0's binary_logloss: 0.490066
[1094]	valid_0's binary_logloss: 0.490068
[1095]	valid_0's binary_logloss: 0.490064
[1096]	valid_0's binary_logloss: 0.490066
[1097]	valid_0's binary_logloss: 0.49006
[1098]	valid_0's binary_logloss: 0.490064
[1099]	valid_0's binary_logloss: 0.490063
[1100]	valid_0's binary_logloss: 0.49006
[1101]	valid_0's binary_logloss: 0.490063
[1102]	valid_0's binary_logloss: 0.49006
[1103]	valid_0's binary_logloss: 0.490057
[1104]	valid_0's binary_logloss: 0.490054
[1105]	valid_0's binary_logloss: 0.490052
[1106]	valid_0's binary_logloss: 0.490056
[1107]	valid_0's binary_logloss: 0.490054
[1108]	valid_0's binary_logloss: 0.490051
[1109]	valid_0's binary_logloss: 0.490052
[1110]	valid_0's binary_logloss: 0.49005
[1111]	valid_0's binary_logloss: 0.490049
[1112]	valid_0's binary_logloss: 0.490047
[1113]	valid_0's binary_logloss: 0.490048
[1114]	valid_0's binary_logloss: 0.490052
[1115]	valid_0's binary_logloss: 0.490051
[1116]	valid_0's binary_logloss: 0.490

[1310]	valid_0's binary_logloss: 0.489907
[1311]	valid_0's binary_logloss: 0.489907
[1312]	valid_0's binary_logloss: 0.489906
[1313]	valid_0's binary_logloss: 0.489907
[1314]	valid_0's binary_logloss: 0.489909
[1315]	valid_0's binary_logloss: 0.48991
[1316]	valid_0's binary_logloss: 0.489907
[1317]	valid_0's binary_logloss: 0.489906
[1318]	valid_0's binary_logloss: 0.489907
[1319]	valid_0's binary_logloss: 0.489903
[1320]	valid_0's binary_logloss: 0.489902
[1321]	valid_0's binary_logloss: 0.489902
[1322]	valid_0's binary_logloss: 0.4899
[1323]	valid_0's binary_logloss: 0.489901
[1324]	valid_0's binary_logloss: 0.4899
[1325]	valid_0's binary_logloss: 0.489902
[1326]	valid_0's binary_logloss: 0.4899
[1327]	valid_0's binary_logloss: 0.489897
[1328]	valid_0's binary_logloss: 0.489896
[1329]	valid_0's binary_logloss: 0.489895
[1330]	valid_0's binary_logloss: 0.489897
[1331]	valid_0's binary_logloss: 0.489896
[1332]	valid_0's binary_logloss: 0.489898
[1333]	valid_0's binary_logloss: 0.489901

datetime.timedelta(seconds=9, microseconds=504185)

In [34]:
#predicting on train set
y_pred_lbg_train = lgbm.predict(X_train_corr)
print('y_pred_lbg_train', y_pred_lbg_train)

#predicting on test set
y_pred_lbg = lgbm.predict(X_test_corr)
print('y_pred_lbg', y_pred_lbg)

y_pred_lbg_train = np.where(y_pred_lbg_train > .5, 1 , 0)
y_pred_lbg = np.where(y_pred_lbg > .5, 1 , 0)

y_pred_lbg_train [0.94864222 0.69442847 0.80277669 ... 0.58135555 0.73334652 0.79187396]
y_pred_lbg [0.7986758  0.9468112  0.80978005 ... 0.84769803 0.82771608 0.71156103]


In [35]:
#calculating accuracy
print('Train accuracy is {0:0.4} ' .format(accuracy_score(y_train,y_pred_lbg_train)))
print('Test accuracy is {0:0.4} ' .format(accuracy_score(y_test,y_pred_lbg)))

Train accuracy is 0.7818 
Test accuracy is 0.7729 


In [36]:
cm = confusion_matrix(y_train, y_pred_lbg_train)
print('Confusion matrix Train Data\n', cm, '\n')

cm = confusion_matrix(y_test, y_pred_lbg)
print('Confusion matrix Validation Data\n', cm)

Confusion matrix Train Data
 [[ 2489 11853]
 [ 1242 44434]] 

Confusion matrix Validation Data
 [[  954  5179]
 [  663 18927]]


## Categorical Data

In [37]:
# Separating Categorical Data
category_data = X.select_dtypes(exclude=[np.number])
category_data.shape

(85741, 19)

In [38]:
category_data.describe()

,x12,x27,x32,x37,x41,x52,x54,x56,x57,x73,x76,x78,x79,x84,x96,x101,x104,x120,x123
count,85741,85686,85741,85739,83171,44237,85366,40599,85741,85456,85739,85741,85741,85739,83171,85741,85741,80620,85741
unique,3,90,3,12,3,35,16163,7,3,22,7,18,9,7,3,4,8,118,5
top,A,BM,A,J,C,G,AGDF,C,B,F,E,C,C,A,A,D,F,BW,E
freq,41738,4319,52749,8343,82973,12123,1767,24111,85165,16185,20246,25847,41611,20246,66265,56266,56268,8502,41414


In [39]:
display_missing_data(category_data)

x12      0.000000
x27      0.064147
x32      0.000000
x37      0.002333
x41      2.997399
x52     48.406247
x54      0.437364
x56     52.649258
x57      0.000000
x73      0.332396
x76      0.002333
x78      0.000000
x79      0.000000
x84      0.002333
x96      2.997399
x101     0.000000
x104     0.000000
x120     5.972639
x123     0.000000
dtype: float64

In [40]:
category_data = category_data.drop(columns=['x52', 'x54', 'x56'],axis=1)

In [41]:
X_train, X_test, y_train, y_test = train_test_split(
    category_data, y, test_size = 0.3, random_state = 8)

X_train.shape, X_test.shape

((60018, 16), (25723, 16))

In [42]:
X_train = X_train.fillna(X_train.mode().iloc[0])
X_test = X_test.fillna(X_train.mode().iloc[0])

X_train.shape, X_test.shape

((60018, 16), (25723, 16))

In [43]:
# Creating Dummy Variable for Training Data
dummy_trn = pd.DataFrame()

for i in category_data:
    dummy_trn = pd.concat([dummy_trn, 
                           pd.get_dummies(data = X_train[i], prefix = i,
                                          drop_first = True)], axis = 1)

In [44]:
# Creating Dummy Variable for Training Data
dummy_tst = pd.DataFrame()

for i in category_data:
    dummy_tst = pd.concat([dummy_tst, 
                           pd.get_dummies(data = X_test[i], prefix = i,
                                          drop_first = True)], axis = 1)

In [45]:
if not (len(dummy_trn.columns) == len(dummy_tst.columns)):
    print(len(dummy_trn.columns), len(dummy_tst.columns))
    print('\ndummy variable length not equal' + '\n' +  'removing uncommom features\n')
    intesection = set(dummy_trn.columns).intersection(set(dummy_tst))
    dummy_trn = dummy_trn[intesection]
    dummy_tst = dummy_tst[intesection]
#     print(len(dummy_trn.columns), len(dummy_tst.columns))
else:
    print('Train and Test Sets have same Dummy Variables')

dummy_trn.shape, dummy_tst.shape

293 277

dummy variable length not equal
removing uncommom features



((60018, 271), (25723, 271))

#### Creating Model for Categorical Data

In [46]:
train_data = lgb.Dataset(dummy_trn, label = y_train)
val_data = lgb.Dataset(dummy_tst, label = y_test)

In [47]:
#setting parameters for lightgbm
param = {'num_leaves':30, 'objective':'binary','max_depth':7,
                 'learning_rate':.05,'max_bin':50, 'metric':'binary_logloss',
                 'verbose': -1, 'lambda_l1': 0.001}

#training our model using light gbm

num_round = 1500
start = datetime.now()
lgbm = lgb.train(param, train_set=train_data, num_boost_round= num_round,
                 valid_sets= val_data, verbose_eval = True, early_stopping_rounds=50)
stop = datetime.now()
execution_time_lgbm = stop-start
execution_time_lgbm

[1]	valid_0's binary_logloss: 0.545184
Training until validation scores don't improve for 50 rounds
[2]	valid_0's binary_logloss: 0.54151
[3]	valid_0's binary_logloss: 0.538193
[4]	valid_0's binary_logloss: 0.535149
[5]	valid_0's binary_logloss: 0.532416
[6]	valid_0's binary_logloss: 0.529895
[7]	valid_0's binary_logloss: 0.52762
[8]	valid_0's binary_logloss: 0.525555
[9]	valid_0's binary_logloss: 0.52363
[10]	valid_0's binary_logloss: 0.521905
[11]	valid_0's binary_logloss: 0.52035
[12]	valid_0's binary_logloss: 0.518892
[13]	valid_0's binary_logloss: 0.517513
[14]	valid_0's binary_logloss: 0.516312
[15]	valid_0's binary_logloss: 0.515214
[16]	valid_0's binary_logloss: 0.514211
[17]	valid_0's binary_logloss: 0.513215
[18]	valid_0's binary_logloss: 0.512338
[19]	valid_0's binary_logloss: 0.511576
[20]	valid_0's binary_logloss: 0.510802
[21]	valid_0's binary_logloss: 0.510081
[22]	valid_0's binary_logloss: 0.509442
[23]	valid_0's binary_logloss: 0.508853
[24]	valid_0's binary_logloss: 0

[270]	valid_0's binary_logloss: 0.496841
[271]	valid_0's binary_logloss: 0.496825
[272]	valid_0's binary_logloss: 0.496832
[273]	valid_0's binary_logloss: 0.496812
[274]	valid_0's binary_logloss: 0.496816
[275]	valid_0's binary_logloss: 0.496826
[276]	valid_0's binary_logloss: 0.496828
[277]	valid_0's binary_logloss: 0.496834
[278]	valid_0's binary_logloss: 0.496839
[279]	valid_0's binary_logloss: 0.496825
[280]	valid_0's binary_logloss: 0.496822
[281]	valid_0's binary_logloss: 0.496831
[282]	valid_0's binary_logloss: 0.496843
[283]	valid_0's binary_logloss: 0.496845
[284]	valid_0's binary_logloss: 0.496846
[285]	valid_0's binary_logloss: 0.496855
[286]	valid_0's binary_logloss: 0.496858
[287]	valid_0's binary_logloss: 0.496841
[288]	valid_0's binary_logloss: 0.496832
[289]	valid_0's binary_logloss: 0.496846
[290]	valid_0's binary_logloss: 0.496833
[291]	valid_0's binary_logloss: 0.496819
[292]	valid_0's binary_logloss: 0.496789
[293]	valid_0's binary_logloss: 0.496747
[294]	valid_0's 

datetime.timedelta(seconds=1, microseconds=265151)

In [48]:
#predicting on train set
y_pred_lbg_train = lgbm.predict(dummy_trn)

#predicting on test set
y_pred_lbg = lgbm.predict(dummy_tst)


In [49]:
y_pred_lbg_train = np.where(y_pred_lbg_train > .5, 1 , 0)
y_pred_lbg = np.where(y_pred_lbg > .5, 1 , 0)

In [50]:
#calculating accuracy
print('Train accuracy is {0:0.4} ' .format(accuracy_score(y_train,y_pred_lbg_train)))
print('Test accuracy is {0:0.4} ' .format(accuracy_score(y_test,y_pred_lbg)))

Train accuracy is 0.7705 
Test accuracy is 0.7645 


# Categorical Plus Numerical Data 

In [51]:
# Concatinating Numerical And Categorical Data

concat_trn = pd.concat([X_train_corr, dummy_trn], axis = 1 )
concat_tst = pd.concat([X_test_corr, dummy_tst], axis = 1 )

concat_trn.shape, concat_tst.shape

((60018, 325), (25723, 325))

In [52]:
# Submission File
    
train_data = lgb.Dataset(concat_trn, label= y_train)
val_data = lgb.Dataset(concat_tst, label= y_test)

#setting parameters for lightgbm
param = {'num_leaves':30, 'objective':'binary','max_depth':7,
         'learning_rate':.05,'max_bin':50, 'metric':'binary_logloss',
         'verbose': -1, 'lambda_l1' : 0.01}

#training our model using light gbm
num_round = 1000
print('[Info]: Model is getting Trained ...')
lgbm = lgb.train(param, train_set = train_data, num_boost_round= num_round,
                 valid_sets = val_data, verbose_eval = True, early_stopping_rounds = 100)

print('[Info]: Model is predicting the Data ... ')        
pred_prob = lgbm.predict(concat_tst)


[Info]: Model is getting Trained ...
[1]	valid_0's binary_logloss: 0.542377
Training until validation scores don't improve for 100 rounds
[2]	valid_0's binary_logloss: 0.536393
[3]	valid_0's binary_logloss: 0.531052
[4]	valid_0's binary_logloss: 0.526336
[5]	valid_0's binary_logloss: 0.52194
[6]	valid_0's binary_logloss: 0.518075
[7]	valid_0's binary_logloss: 0.514444
[8]	valid_0's binary_logloss: 0.511131
[9]	valid_0's binary_logloss: 0.508126
[10]	valid_0's binary_logloss: 0.505472
[11]	valid_0's binary_logloss: 0.502956
[12]	valid_0's binary_logloss: 0.50077
[13]	valid_0's binary_logloss: 0.498613
[14]	valid_0's binary_logloss: 0.496747
[15]	valid_0's binary_logloss: 0.494943
[16]	valid_0's binary_logloss: 0.493331
[17]	valid_0's binary_logloss: 0.491814
[18]	valid_0's binary_logloss: 0.490436
[19]	valid_0's binary_logloss: 0.489184
[20]	valid_0's binary_logloss: 0.487993
[21]	valid_0's binary_logloss: 0.486873
[22]	valid_0's binary_logloss: 0.485794
[23]	valid_0's binary_logloss: 0

[217]	valid_0's binary_logloss: 0.468179
[218]	valid_0's binary_logloss: 0.468185
[219]	valid_0's binary_logloss: 0.468169
[220]	valid_0's binary_logloss: 0.468153
[221]	valid_0's binary_logloss: 0.468146
[222]	valid_0's binary_logloss: 0.468127
[223]	valid_0's binary_logloss: 0.46815
[224]	valid_0's binary_logloss: 0.468135
[225]	valid_0's binary_logloss: 0.468152
[226]	valid_0's binary_logloss: 0.468136
[227]	valid_0's binary_logloss: 0.468127
[228]	valid_0's binary_logloss: 0.468102
[229]	valid_0's binary_logloss: 0.468101
[230]	valid_0's binary_logloss: 0.46812
[231]	valid_0's binary_logloss: 0.468152
[232]	valid_0's binary_logloss: 0.468152
[233]	valid_0's binary_logloss: 0.468116
[234]	valid_0's binary_logloss: 0.468113
[235]	valid_0's binary_logloss: 0.468124
[236]	valid_0's binary_logloss: 0.468111
[237]	valid_0's binary_logloss: 0.468097
[238]	valid_0's binary_logloss: 0.46811
[239]	valid_0's binary_logloss: 0.468087
[240]	valid_0's binary_logloss: 0.468074
[241]	valid_0's bin

In [53]:
pred_prob_trn = lgbm.predict(concat_trn)

In [54]:
pred_prob_trn = np.where(pred_prob_trn > .5, 1 , 0)
pred_prob = np.where(pred_prob > .5, 1 , 0)

In [55]:
#calculating accuracy
print('Train accuracy = {0:0.4} ' .format(accuracy_score(y_train,pred_prob_trn)))
print('Test accuracy = {0:0.4} ' .format(accuracy_score(y_test,pred_prob)))

Train accuracy = 0.8025 
Test accuracy = 0.7811 
